In [1]:
import os
import sys


# IMPORTANT: Set these BEFORE importing torch or any ML libraries
# Disable all GPU/MPS backends to prevent crashes with IFD analyzer
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Disable CUDA
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"  # Disable MPS memory allocation
os.environ["DISABLE_MPS_COMPAT"] = "1"  # Additional MPS disable flag
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"  # Disable HuggingFace telemetry
os.environ["TRANSFORMERS_OFFLINE"] = "0"  # Allow model downloads

# Force CPU usage in PyTorch to avoid MPS crashes
import torch

# Forcefully disable MPS before anything else
torch.set_default_device("cpu")
if hasattr(torch.backends, "mps"):
    # Monkey-patch to prevent MPS usage
    original_is_available = torch.backends.mps.is_available
    torch.backends.mps.is_available = lambda: False
    print("MPS has been disabled - forcing CPU-only mode")
else:
    print("Using CPU for all computations")

print(f"PyTorch device: {torch.get_default_device()}")
print(f"PyTorch version: {torch.__version__}")

MPS has been disabled - forcing CPU-only mode
PyTorch device: cpu
PyTorch version: 2.6.0


In [2]:
import os
from oumi.core.configs import AnalyzeConfig
from oumi.core.analyze.dataset_analyzer import DatasetAnalyzer

# Path to the config file
config_path = "/Users/ryanarman/code/oumi/configs/examples/analyze/analyze.yaml"

# Path to your dataset file
dataset_path = "/Users/ryanarman/code/scratch/ryan_hillclimbing_experiments/banking77/notebooks/data/banking77_train.jsonl"

# Load the config from YAML
config = AnalyzeConfig.from_yaml(
    config_path=config_path,
)

config.sample_count = 10

# Override the dataset settings to use your local file
config.dataset_path = dataset_path
config.dataset_name = None  # Clear dataset_name so it uses dataset_path instead

# Optionally update output path
config.output_path = "./analysis_output/banking77"

# IMPORTANT: Disable analyzers that require large model downloads or have issues
# IFD requires downloading Qwen model and may cause MPS crashes
# fasttext requires additional dependencies
# repr_diversity and question_diversity download embedding models
print(f"Running {len(config.analyzers)} analyzers: {[a.id for a in config.analyzers]}")

# Validate the configuration
config.finalize_and_validate()

# Create the analyzer
analyzer = DatasetAnalyzer(config)

Running 18 analyzers: ['length', 'diversity', 'format', 'quality', 'training_quality', 'cost', 'content_pattern', 'question_diversity', 'fasttext', 'ifd', 'repr_diversity', 'task_category', 'safety', 'difficulty', 'instruct_reward', 'input_quality', 'conversation_structure', 'response_completeness']
[2025-12-29 17:30:15,181][oumi][rank0][pid:93611][MainThread][INFO]][base_map_dataset.py:91] Creating map dataset (type: TextSftJsonLinesDataset)... dataset_name: 'custom'
[2025-12-29 17:30:15,181][oumi.utils.analysis_utils][rank0][pid:93611][MainThread][INFO]][analysis_utils.py:225] Loaded text dataset from: /Users/ryanarman/code/scratch/ryan_hillclimbing_experiments/banking77/notebooks/data/banking77_train.jsonl
[2025-12-29 17:30:15,182][oumi][rank0][pid:93611][MainThread][INFO]][dataset_analyzer.py:154] Loaded dataset from config: None
[2025-12-29 17:30:15,311][oumi][rank0][pid:93611][MainThread][INFO]][dataset_analyzer.py:304] Initialized sample analyzer: length
[2025-12-29 17:30:15,312

In [3]:
# Run the analysis
analyzer.analyze_dataset()

# The results are stored in analyzer object
if analyzer._analysis_results:
    print(
        f"Total conversations analyzed: {analyzer._analysis_results.conversations_analyzed}"
    )

[2025-12-29 17:30:15,345][oumi][rank0][pid:93611][MainThread][INFO]][dataset_analyzer.py:347] Starting analysis of dataset: None
[2025-12-29 17:30:15,346][oumi][rank0][pid:93611][MainThread][INFO]][dataset_analyzer.py:348] Using 18 sample analyzers: ['length', 'diversity', 'format', 'quality', 'training_quality', 'cost', 'content_pattern', 'question_diversity', 'fasttext', 'ifd', 'repr_diversity', 'task_category', 'safety', 'difficulty', 'instruct_reward', 'input_quality', 'conversation_structure', 'response_completeness']
[2025-12-29 17:30:15,347][oumi][rank0][pid:93611][MainThread][INFO]][dataset_analyzer.py:371] Analyzing 10 of 8002 conversations
[2025-12-29 17:30:15,348][oumi][rank0][pid:93611][MainThread][INFO]][dataset_analyzer.py:421] Converting conversation dataset with 8002 items
[2025-12-29 17:30:15,348][oumi][rank0][pid:93611][MainThread][INFO]][dataset_analyzer.py:428] Limiting analysis to first 10 items (dataset has 8002 total)


Converting Unknown Dataset to DataFrames: 100%|██████████| 10/10 [00:00<00:00, 1351.74item/s]


[2025-12-29 17:30:15,412][oumi][rank0][pid:93611][MainThread][INFO]][question_diversity_analyzer.py:464] Computing embeddings for 10 user questions...
[2025-12-29 17:30:15,413][oumi][rank0][pid:93611][MainThread][INFO]][question_diversity_analyzer.py:174] Loading embedding model: all-MiniLM-L6-v2


Computing embeddings: 100%|██████████| 10/10 [00:00<00:00, 104.13it/s]


[2025-12-29 17:30:16,488][oumi][rank0][pid:93611][MainThread][INFO]][question_diversity_analyzer.py:469] Clustering 10 questions using dbscan...
[2025-12-29 17:30:16,722][oumi][rank0][pid:93611][MainThread][INFO]][question_diversity_analyzer.py:487] Found 1 clusters
[2025-12-29 17:30:16,748][oumi][rank0][pid:93611][MainThread][INFO]][fasttext_analyzer.py:220] Initialized fast-langdetect for language detection
[2025-12-29 17:30:16,749][oumi][rank0][pid:93611][MainThread][INFO]][fasttext_analyzer.py:436] Analyzing language for column: conversation_text_content
[2025-12-29 17:30:16,824][oumi][rank0][pid:93611][MainThread][INFO]][ifd_analyzer.py:154] Loading model for IFD analysis: Qwen/Qwen3-0.6B
[2025-12-29 17:30:18,659][oumi][rank0][pid:93611][MainThread][INFO]][ifd_analyzer.py:193] Loaded Qwen/Qwen3-0.6B on cpu (dtype: torch.float32)
[2025-12-29 17:30:18,661][oumi][rank0][pid:93611][MainThread][INFO]][ifd_analyzer.py:664] Computing IFD scores using instruction='conversation_text_conten

...

[2025-12-29 17:30:21,613][oumi][rank0][pid:93611][MainThread][INFO]][question_diversity_analyzer.py:469] Clustering 10 questions using dbscan...
[2025-12-29 17:30:21,624][oumi][rank0][pid:93611][MainThread][INFO]][question_diversity_analyzer.py:482] Found 0 clusters, 10 unique/diverse questions (not similar to others)
[2025-12-29 17:30:21,625][oumi][rank0][pid:93611][MainThread][INFO]][fasttext_analyzer.py:436] Analyzing language for column: text_content
[2025-12-29 17:30:21,634][oumi][rank0][pid:93611][MainThread][INFO]][ifd_analyzer.py:643] Detected conversation format. Computing IFD for assistant messages using preceding user messages as instructions.
[2025-12-29 17:30:24,259][oumi][rank0][pid:93611][MainThread][INFO]][ifd_analyzer.py:521] IFD analysis complete. Processed 10 assistant messages out of 10 total.
[2025-12-29 17:30:24,260][oumi][rank0][pid:93611][MainThread][INFO]][repr_diversity_analyzer.py:363] Computing diversity scores for 30 samples in column 'text_content'...


Computing embeddings: 100%|██████████| 30/30 [00:00<00:00, 158.95it/s]


[2025-12-29 17:30:24,451][oumi][rank0][pid:93611][MainThread][INFO]][repr_diversity_analyzer.py:230] Computing nearest neighbor distances for 30 samples (k=5)...
[2025-12-29 17:30:24,456][oumi][rank0][pid:93611][MainThread][INFO]][repr_diversity_analyzer.py:556] Column 'text_content': 19/30 samples (63.3%) are redundant


/Users/ryanarman/miniconda3/envs/oumi/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/ryanarman/miniconda3/envs/oumi/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


Total conversations analyzed: 10


In [4]:
schema = analyzer.get_schema()
print(f"len(schema): {len(schema)}")
merged_columns = analyzer.analysis_df.columns
print(f"len(merged_columns): {len(merged_columns)}")

len(schema): 184
len(merged_columns): 184


In [5]:
analyzer.analysis_df.head()

,conversation_index,conversation_id,num_messages,conversation_text_content,conversation_text_content__length__token_count,conversation_text_content__diversity__unique_words_ratio,conversation_text_content__format__has_markdown,conversation_text_content__format__has_json,conversation_text_content__format__has_code_blocks,conversation_text_content__format__code_block_count,...,conversation__conversation_structure__conversation_depth,conversation__conversation_structure__role_balance,conversation__conversation_structure__has_system_prompt,conversation__conversation_structure__avg_turn_length,conversation__conversation_structure__turn_length_variance,text_content__response_completeness__is_complete,text_content__response_completeness__score,text_content__response_completeness__ends_naturally,text_content__response_completeness__has_conclusion,text_content__response_completeness__truncation_type
0,0,0,3,SYSTEM: You are a banking intent classifier. C...,1775,0.516556,True,False,False,0,...,1,0.5,True,6.5,30.25,None,NaN,None,None,None
1,0,0,3,SYSTEM: You are a banking intent classifier. C...,1775,0.516556,True,False,False,0,...,1,0.5,True,6.5,30.25,None,NaN,None,None,None
2,0,0,3,SYSTEM: You are a banking intent classifier. C...,1775,0.516556,True,False,False,0,...,1,0.5,True,6.5,30.25,False,0.5,False,False,
3,1,1,3,SYSTEM: You are a banking intent classifier. C...,1774,0.511602,True,False,False,0,...,1,0.5,True,6.0,25.00,None,NaN,None,None,None
4,1,1,3,SYSTEM: You are a banking intent classifier. C...,1774,0.511602,True,False,False,0,...,1,0.5,True,6.0,25.00,None,NaN,None,None,None


# Conv level

In [6]:
conv_columns = analyzer.conversation_df.columns
row = analyzer.conversation_df.iloc[0]
col = conv_columns[44]
print(f"col: {col}")
print(f"schema[col]: {schema[col]}")
print(f"row[col]: {row[col]}")

col: conversation_text_content__question_diversity__is_concentrated
schema[col]: {'type': <ColumnType.BOOL: 'bool'>, 'content_type': <ContentType.BOOLEAN: 'boolean'>, 'description': 'Whether question is in a concentrated cluster (>threshold)'}
row[col]: True


In [7]:
for i in range(len(conv_columns)):
    if "question" in conv_columns[i]:
        print(i)

42
43
44


# Message level

In [8]:
msg_columns = analyzer.message_df.columns

for i in range(len(msg_columns)):
    if "question" in msg_columns[i]:
        print(i)


44
45
46


In [9]:
row = analyzer.message_df.iloc[1]
col = msg_columns[46]
print(f"col: {col}")
print(f"schema[col]: {schema[col]}")
print(f"row[col]: {row[col]}")

col: text_content__question_diversity__is_concentrated
schema[col]: {'type': <ColumnType.BOOL: 'bool'>, 'content_type': <ContentType.BOOLEAN: 'boolean'>, 'description': 'Whether question is in a concentrated cluster (>threshold)'}
row[col]: False


In [10]:
msg_columns

Index(['conversation_index', 'conversation_id', 'message_index', 'message_id',
       'role', 'text_content', 'text_content__length__token_count',
       'text_content__diversity__unique_words_ratio',
       'text_content__format__has_markdown', 'text_content__format__has_json',
       'text_content__format__has_code_blocks',
       'text_content__format__code_block_count',
       'text_content__format__code_block_languages',
       'text_content__format__has_urls', 'text_content__format__has_emails',
       'text_content__format__format_complexity_score',
       'text_content__quality__has_pii', 'text_content__quality__pii_types',
       'text_content__quality__pii_count',
       'text_content__quality__has_encoding_issues',
       'text_content__quality__repetition_ratio',
       'text_content__quality__has_high_repetition',
       'text_content__training_quality__response_completeness_score',
       'text_content__training_quality__has_proper_ending',
       'text_content__training_